This notebook does not prep the data.

In [1]:
#Imports
import numpy as np
import pandas as pd
import os
import shutil
import glob
import itertools
import random
import matplotlib.pyplot as plt
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner
from keras_tuner import RandomSearch
from sklearn.metrics import confusion_matrix
from PIL import Image

In [2]:
#Preferences
warnings.simplefilter(action="ignore", category=FutureWarning)
%matplotlib inline

In [3]:
#Path variables
train_path = "data/train/"
val_path = "data/val/"
test_path = "data/test/"

In [4]:
path_list = [train_path, val_path, test_path]

In [5]:
datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = datagen.flow_from_directory(directory=train_path, target_size=(256,256), batch_size=32, color_mode="grayscale", classes=["NORMAL", "viral_pneumonia", "bacterial_pneumonia"], save_format="jpeg")

Found 5097 images belonging to 3 classes.


In [7]:
val_generator = datagen.flow_from_directory(directory=val_path, target_size=(256,256), batch_size=32, color_mode="grayscale", classes=["NORMAL", "viral_pneumonia", "bacterial_pneumonia"], save_format="jpeg")

Found 135 images belonging to 3 classes.


In [8]:
test_generator = datagen.flow_from_directory(directory=test_path, target_size=(256,256), batch_size=32, color_mode="grayscale", classes=["NORMAL", "viral_pneumonia", "bacterial_pneumonia"], save_format="jpeg", shuffle=False)

Found 624 images belonging to 3 classes.


In [9]:
img, labels = next(train_generator)

In [10]:
def build_model(hp):
    """
    Building a model to optimize hypter parameters.
    """
    model = Sequential()
    model.add(Conv2D(filters=hp.Int("conv_1_filter", min_value=32, max_value=128, step=16), kernel_size=hp.Choice("conv_1_kernel", values = [3,5]), activation="relu", input_shape=(256,256,1)))
    model.add(MaxPool2D(pool_size=2))
    model.add(Conv2D(filters=hp.Int("conv_1_filter", min_value=32, max_value=64, step=16), kernel_size=hp.Choice("conv_2_kernel", values = [3,5]), activation="relu"))
    model.add(MaxPool2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=hp.Int("dense_1_units", min_value=32, max_value=128, step=16), activation="relu"))
    model.add(Dense(units=3, activation="softmax"))

    model.compile(optimizer=Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3])), loss="categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.Recall()])

    return model

In [11]:
tuner_search0 = RandomSearch(build_model, objective="val_accuracy", max_trials=5, directory="output", project_name="ChestXrayPneumoniaAccuracy")

2023-08-01 11:38:52.328695: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
tuner_search1 = RandomSearch(build_model, objective=keras_tuner.Objective("val_recall", direction="min"), max_trials=5, directory="output", project_name="ChestXrayPneumoniaRecall")

In [13]:
tuner_search0.search(train_generator, epochs=3, validation_data=val_generator)

Trial 5 Complete [00h 11m 16s]
val_accuracy: 0.5185185074806213

Best val_accuracy So Far: 0.6518518328666687
Total elapsed time: 01h 12m 59s
INFO:tensorflow:Oracle triggered exit


In [14]:
tuner_search1.search(train_generator, epochs=3, validation_data=val_generator)

Trial 5 Complete [00h 25m 46s]
val_recall: 0.6888889074325562

Best val_recall So Far: 0.0
Total elapsed time: 03h 48m 00s
INFO:tensorflow:Oracle triggered exit


In [16]:
model0 = tuner_search0.get_best_models(num_models=1)[0]

In [ ]:
model0.summary()

In [17]:
model1 = tuner_search1.get_best_models(num_models=1)[0]

In [ ]:
model1.summary()

In [19]:
model0.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
160/160 [==============================] - 253s 2s/step - loss: 0.5829 - accuracy: 0.7555 - recall: 0.7181 - val_loss: 0.8309 - val_accuracy: 0.6296 - val_recall: 0.5778
Epoch 2/10
160/160 [==============================] - 230s 1s/step - loss: 0.5477 - accuracy: 0.7652 - recall: 0.7406 - val_loss: 0.9381 - val_accuracy: 0.6815 - val_recall: 0.6222
Epoch 3/10
160/160 [==============================] - 208s 1s/step - loss: 0.4409 - accuracy: 0.8115 - recall: 0.7924 - val_loss: 0.7269 - val_accuracy: 0.7333 - val_recall: 0.7111
Epoch 4/10
160/160 [==============================] - 262s 2s/step - loss: 0.3365 - accuracy: 0.8552 - recall: 0.8468 - val_loss: 0.7431 - val_accuracy: 0.7630 - val_recall: 0.7481
Epoch 5/10
160/160 [==============================] - 284s 2s/step - loss: 0.2476 - accuracy: 0.8974 - recall: 0.8925 - val_loss: 0.9234 - val_accuracy: 0.7556 - val_recall: 0.7407
Epoch 6/10
160/160 [==============================] - 219s 1s/step - loss: 0.1834 - accuracy: 0

In [ ]:
model1.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
160/160 [==============================] - 2589s 16s/step - loss: 1.0478 - accuracy: 0.4905 - recall: 0.0995 - val_loss: 1.1940 - val_accuracy: 0.2815 - val_recall: 0.2815
Epoch 2/10
160/160 [==============================] - 2586s 16s/step - loss: 1.0474 - accuracy: 0.4905 - recall: 0.1575 - val_loss: 1.1856 - val_accuracy: 0.2815 - val_recall: 0.0000e+00
Epoch 3/10
160/160 [==============================] - 2587s 16s/step - loss: 1.0471 - accuracy: 0.4905 - recall: 0.0906 - val_loss: 1.1919 - val_accuracy: 0.2815 - val_recall: 0.0000e+00
Epoch 4/10
160/160 [==============================] - 2736s 17s/step - loss: 1.0473 - accuracy: 0.4905 - recall: 0.0708 - val_loss: 1.1819 - val_accuracy: 0.2815 - val_recall: 0.0000e+00
Epoch 5/10
160/160 [==============================] - 2591s 16s/step - loss: 1.0472 - accuracy: 0.4905 - recall: 0.1265 - val_loss: 1.1744 - val_accuracy: 0.2815 - val_recall: 0.0000e+00
Epoch 6/10
160/160 [==============================] - 2556s 16s/step 

#Building a CNN
model = Sequential([
    Conv2D(filters = 32, kernel_size=(3,3), activation="relu", padding="same", input_shape=(256,256,1)),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters = 64, kernel_size=(3,3), activation="relu", padding="same"),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters = 128, kernel_size=(3,3), activation="relu", padding="same"),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=128, activation="relu"),
    Dense(units=3, activation="softmax")
])

#Compiling the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.Recall()])

model.fit(x=train_generator, validation_data=val_generator, epochs=10, verbose=2)

### Make a prediction

In [ ]:
test_imgs, test_labels = next(test_generator)
plotImages(test_imgs)
print(test_labels)

In [ ]:
predictions0 = model0.predict(x=test_generator, verbose=0)

In [ ]:
predictions1 = model1.predict(x=test_generator, verbose=0)

In [ ]:
cm0 = confusion_matrix(y_true=test_generator.classes, y_pred=np.argmax(predictions0, axis=-1))

In [ ]:
cm1 = confusion_matrix(y_true=test_generator.classes, y_pred=np.argmax(predictions1, axis=-1))

In [ ]:
#Confusion matrix plot function from TensorFlows website
def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.Blues):
    """
    A function to plot results in a confusion matrix.
    """
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i,j], horizontalalignment="center", color = "white" if cm[i,j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [ ]:
cm_plot_labels = ["Normal","viral_pneumonia", "bacterial_pneumonia"]
plot_confusion_matrix(cm=cm0, classes=cm_plot_labels, title = "Confusion Matrix")

In [ ]:
plot_confusion_matrix(cm=cm1, classes=cm_plot_labels, title = "Confusion Matrix")